In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np
from plotly.subplots import make_subplots

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/ds

/content/drive/MyDrive/ds


# Score Ranges Flow

In [4]:
def add_label(data_path, ranges, colors, subject):
  #divide scores into ranges
  province_avg = pd.read_csv(data_path)
  criteria = [province_avg[subject].between(r[0], r[1]) for r in ranges]
  province_avg['Level'] = np.select(criteria, [i for i in range(len(ranges))], 0)
  province_avg['Color'] = np.select(criteria, colors, 0)

  return province_avg[['CityCode', subject, 'Level', 'Color']]

In [7]:
def sankey(ranges, nodes, node_colors, path_colors, subject, weight=False):
  """
  ranges: list of scores ranges, must not be overlap and covers of scores from 0-10
  nodes: list of ranges name
  node_colors: list of colors of nodes
  path_color: list of colors of links
  subject: subject to display
  weight: boolean values, define whether the links are weighted or not
  """
  province_avg2021 = add_label("/content/drive/MyDrive/ds/Province_Avg2021.csv", ranges, path_colors, subject)
  province_avg2020 = add_label("/content/drive/MyDrive/ds/Province_Avg2020.csv", ranges, path_colors, subject)
  province_avg2019 = add_label("/content/drive/MyDrive/ds/Province_Avg2019.csv", ranges, path_colors, subject)
  province_avg2018 = add_label("/content/drive/MyDrive/ds/Province_Avg2018.csv", ranges, path_colors, subject)

  if weight == False:
    value = [5]*189
  else:
    value = list(province_avg2019[subject] - province_avg2018[subject]) + list(province_avg2020[subject] - province_avg2019[subject]) + list(province_avg2021[subject] - province_avg2020[subject])
  
  fig = go.Figure(data=[go.Sankey(
    arrangement = "snap",
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "rgba(255, 0, 0, 0)", width = 0.5),
      label = nodes*4,
      color = node_colors*4,
      # x= [0]*5 + [0.25]*5 + [0.5]*5 + [0.75]*5,
      # y= [0, 0.2, 0.4, 0.6, 0.8]*4,
    ),
    link = dict(
      source = list(province_avg2018['Level'])+ list(province_avg2019['Level'] +len(ranges)) + list(province_avg2020['Level'] + 2*len(ranges)), # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = list(province_avg2019['Level'] +len(ranges)) + list(province_avg2020['Level'] +2*len(ranges)) +list(province_avg2021['Level'] +3*len(ranges)),
      value = value,
      color = list(province_avg2018['Color'])+ list(province_avg2019['Color']) + list(province_avg2020['Color']),
  ))])
  fig.update_layout(title_text=subject, font_size=10)
  fig.update_layout(width=1000)
  fig.update_layout(height=700)
  fig.show()

In [8]:
sankey(ranges=[(0, 4.5), (4.5, 5.5), (5.5, 6.5), (6.5, 7), (7, 10)],
       nodes = ["0 - 4.5", "4.5 - 5.5", "5.5 - 6.5", "6.5 - 7",  "7 - 10"],
       node_colors = ["rgba(215, 47, 47, 1)", "rgba(215, 136, 47, 1)", "rgba(215, 209, 47, 1)", "rgba(150, 215, 47, 1)", "rgba(47, 215, 84, 1)"],
       path_colors = ["rgba(215, 47, 47, 0.5)", "rgba(215, 136, 47, 0.5)", "rgba(215, 209, 47, 0.5)", "rgba(150, 215, 47, 0.5)", "rgba(47, 215, 84, 0.5)"],
       subject="Math",
  )